# Introduction

This notebook contains the algorithm of using finite difference methods to solve Black-Scholes Equation in order to get predictions for future option price.

Main reference: *An ill-posed problem for the Black-Scholes equation for a profitable forecast of prices of stock options on real market data*

### Current Problem
The minizers using finite diff scheme -> tikhonov reg are tending to zero, regardless of the option/stock price.

In [9]:
from eq_solver import num_solver as ns

# This data comes from 531data/p1.csv:AAL US 1/18/2019 C40 Equity
block = ns.DataBlock(today='10/19/2016',
                  option_ask = [8.44999981, 8.55000019, 9.10000038],
                  option_bid = [7.05000019, 7.8499999, 8.5],
                  volatility = [39.456, 38.061, 37.096],
                  stock_ask = 40.66,
                  stock_bid = 40.65)

block.af_system(3)

print('Af system is:', block.af)

block.reg(3, 1)

Af system is: (array([[ 1.13714669e+10, -2.27429338e+10,  1.13714669e+10,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.11111111e-01,  0.00000000e+00],
       [ 0.00000000e+00, -1.11111111e-01,  0.00000000e+00,
         2.42455418e+10, -4.84910835e+10,  2.42455418e+10,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000

array([ 1.06140557e-32, -2.53620900e-32,  1.06140557e-32,  9.02541335e-22,
       -1.80508267e-21,  9.02541335e-22,  0.00000000e+00, -1.03701560e-43,
        0.00000000e+00])

## Problem

As you can see from the code above, the result from tikhonov regularization is incredibly small. 

In order to solve the problem, I found the following code interesting:

In [30]:
# Here's the test for the tikhonov regularization.
import numpy as np

scale = 1
x = np.array([[scale, scale, scale], [0, 1, 0], [0, 0, 1]])
f = np.array([0, 4, 8])
print('The true solution is:')
print(ns.tikhonov(x, f, 0))
print()

while scale < 10 ** 10:
    x = np.array([[scale, scale, scale], [0, 1, 0], [0, 0, 1]])

    print(f'Tikhonov result with beta = 0.01 and scale = {scale}:')
    print(ns.tikhonov(x, f, 0.01))
    print()
    scale *= 10

The true solution is:
[-12.   4.   8.]

Tikhonov result with beta = 0.01 and scale = 1:
[-11.53735218   3.84616483   7.80656087]

Tikhonov result with beta = 0.01 and scale = 10:
[-11.64934312   3.84505601   7.80545205]

Tikhonov result with beta = 0.01 and scale = 100:
[-11.65047401   3.84504481   7.80544085]

Tikhonov result with beta = 0.01 and scale = 1000:
[-11.65048532   3.8450447    7.80544074]

Tikhonov result with beta = 0.01 and scale = 10000:
[-11.65048447   3.84504423   7.80544025]

Tikhonov result with beta = 0.01 and scale = 100000:
[-11.65053463   3.84506974   7.80546488]

Tikhonov result with beta = 0.01 and scale = 1000000:
[-11.65173268   3.84568747   7.80604521]

Tikhonov result with beta = 0.01 and scale = 10000000:
[-11.24438268   3.65296057   7.59142211]

Tikhonov result with beta = 0.01 and scale = 100000000:
[1.33333333e-16 1.33333333e-16 1.33333333e-16]

Tikhonov result with beta = 0.01 and scale = 1000000000:
[1.33333333e-18 1.33333333e-18 1.33333333e-18]



### Explanation:
From the example shown above, we can conclude that even if two matrices are mathematically equivalent, such as

$$ X = \begin{bmatrix}
 1 & 1 & 1\\
 0 & 1 & 0\\
 0 & 0 & 1
\end{bmatrix}$$

and 

$$ X' = \begin{bmatrix}
 100000 & 100000 & 100000\\
 0 & 1 & 0\\
 0 & 0 & 1
\end{bmatrix};$$


are not necessarily equivalent in the tikhonov regularization.

It is because of the tikhonov reg parameter treats every row equivalently, as in the solution

$$ U = (\mathcal{A}^T A + \beta I_{MN})^{-1} \mathcal{A}^T f.$$

However, in the `num_solver`, the matrix is having incredibly large indices from the Black Scholes Equation as we can see from the results above (like `[2.42455418e+10, -4.84910835e+10,  2.42455418e+10]`).

### Potential Solution
Normalize each row vector might be a feasible way. For example, the matrix $X'$ will be normalized to

$$ X'_n = \begin{bmatrix}
 \sqrt{1/3} & \sqrt{1/3} & \sqrt{1/3} \\
 0 & 1 & 0\\
 0 & 0 & 1
\end{bmatrix};$$

Now we can test on this normalized matrix.

In [26]:
# Test X'_n.

import math

scale = math.sqrt(1/3)
x = np.array([[scale, scale, scale], [0, 1, 0], [0, 0, 1]])
f = np.array([0, 4, 8])
print(f'New tikhonov result with beta = 0.01 and scale = {scale}:')
print(ns.tikhonov(x, f, 0.01))
print('While the true solution is')
print(ns.tikhonov(x, f, 0))

New tikhonov result with beta = 0.01 and scale = 0.5773502691896257:
[-11.31755164   3.84834107   7.80873711]
While the true solution is
[-12.   4.   8.]
